In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut, GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [19]:
df = pd.read_csv("../Data/student-por.csv")
display(df)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


Undersøker korrelasjonen mellom variablene:

In [3]:
#pd.plotting.scatter_matrix(df, figsize=(20,20))
#plt.show()

Sjekker om dataene inneholder tomme 

In [4]:
df.isnull().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

In [5]:
categorical_cols=["sex", "school", "address", "Pstatus", "Mjob", "Fjob", "guardian", "famsize", "reason", "schoolsup", "famsup", "activities", "paid", "internet", "nursery", "higher", "romantic"]
#famrel, fedu, medu, traveltime, studytime, failures(maks 4), freetime, goout, Walc, Dalc, health, (G1, G2)
df = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols, prefix_sep="_", drop_first = True)


In [6]:
#Model 1
X_1 = df.drop(["G3", ], axis=1)

#Model 2
X_2 = df[["G1", "G2"]]

#Model 3
#X_3 = []

#Response
Y = df["G3"]

X_1_train, X_1_test, X_2_train, X_2_test, Y_train, Y_test = train_test_split(X_1, X_2, Y, test_size = 0.20)

In [7]:
#Model 1 trening:
loo = LeaveOneOut()
loo.get_n_splits(X_1_train)
print(loo)

residuals = []
for train_index, test_index in loo.split(X_1_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    loo_train_x, loo_test_x = X_1_train.iloc[train_index], X_1_train.iloc[test_index]
    loo_train_y, loo_test_y = Y_train.iloc[train_index], Y_train.iloc[test_index]
    lm = LinearRegression().fit(loo_train_x, loo_train_y)
    residuals.append(lm.predict(loo_test_x) - loo_test_y)

LeaveOneOut()


In [10]:
#print(residuals)
#MSE = (np.asarray(residuals)**2).mean()
#display(MSE)
#print(Y_train.var())
#np.argmax(residuals)

[354   -1.817244
Name: G3, dtype: float64, 323    1.031328
Name: G3, dtype: float64, 304   -0.621713
Name: G3, dtype: float64, 415   -0.954911
Name: G3, dtype: float64, 572   -1.490098
Name: G3, dtype: float64, 295    0.401302
Name: G3, dtype: float64, 582    0.930295
Name: G3, dtype: float64, 284    1.281895
Name: G3, dtype: float64, 239    0.157717
Name: G3, dtype: float64, 524    1.299575
Name: G3, dtype: float64, 482    0.976585
Name: G3, dtype: float64, 55    1.233074
Name: G3, dtype: float64, 550    0.269153
Name: G3, dtype: float64, 399   -1.240204
Name: G3, dtype: float64, 322   -2.218184
Name: G3, dtype: float64, 432   -1.989624
Name: G3, dtype: float64, 33    0.867733
Name: G3, dtype: float64, 463   -1.296379
Name: G3, dtype: float64, 329   -1.459142
Name: G3, dtype: float64, 266    1.136044
Name: G3, dtype: float64, 617   -0.241644
Name: G3, dtype: float64, 376    0.610316
Name: G3, dtype: float64, 259    1.612763
Name: G3, dtype: float64, 209   -0.139206
Name: G3, dtype: fl

KeyError: 0

In [28]:


# Parametrene man vil teste ut all kombinasjoner av:
params = {"eta": [1, 0.1, 0.001], "max_depth": [2, 3, 4, 5, 6]}
# Modellen man vil prøve ut parametrene på. Defineres med ingen parametre (tom parantes)
xgb_model = xgb.XGBRegressor()
# Tester modellen på alle parametrene
GsCV = GridSearchCV(xgb_model, params, verbose=0, scoring="neg_mean_squared_error", cv=5)
GsCV.fit(X_1_train, Y_train)
# Henter ut modellen med de parametrene som gav best score, og andre interessante ting
model = GsCV.best_estimator_
print(GsCV.best_score_)
print(GsCV.best_params_)

-1.8606810862649859
{'eta': 0.1, 'max_depth': 2}


In [ ]:
print(model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
